In [1]:
%qtconsole

In [4]:
import pandas as pd
import os
import re
import romkan
import unicodedata
import csv

In [5]:
# get irregular words from ../data/irregular/irregular.csv and create a dict to avoid those words

In [11]:
kanji[kanji.character in ['広','大']]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [7]:
propernouns = pd.read_csv('../data/propernouns_subset.csv')
kanji = pd.read_csv('../data/kanjidic2.csv')

In [14]:
#remove all entries with Roman characters and numbers in the name
propernouns = propernouns[propernouns.kanji.map(lambda x: True if re.match('[０-９]+|[ａ-ｚＡ-Ｚ]+|ゝ|〆', x) == None else False)]

In [6]:
# Run once
del propernouns['index']
del kanji['Unnamed: 0']
propernouns.pronunciation = propernouns.pronunciation.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
kanji.on = kanji.on.map(lambda x: re.findall("""[\u3040-\u309f]+""", romkan.to_hiragana(romkan.to_roma(x))))
##############################
#kanji.kun = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x))))) # THIS THREW AWAY GOOD INFO
##############################
kanji['kun1'] = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x)))))
kanji['kun2'] = kanji.kun.map(lambda x: list(set(map(lambda y: ''.join(y.split('.')),re.findall("""[\u3040-\u309f.]+""", x)))))
# I should also include the kun without splitting somehow... aka shi.ku = shi and shiku
kanji.nanori = kanji.nanori.map(lambda x: re.findall("""[\u3040-\u309f]+""", x))
kanji.meaning = kanji.meaning.map(lambda x: re.findall("""[a-zA-Z-]+ *[a-zA-Z-]+""", x))
#char_ser = pd.Series(kanji.on.values,index=kanji.character).to_dict()
kanji['onkun'] = kanji.kun1 + kanji.kun2 + kanji.on
#kanji.onkun.map(lambda x: list(set(x))) ########## NOW MAKE SURE ALL ARE SET TO LISTS SO THERE ARE NO DUPLICATES
char_ser = pd.Series(kanji.onkun.values,index=kanji.character).to_dict()
kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
0,亜,1509.0,"[Asia, rank next, come after, -ous]",[あ],['つ.ぐ'],"[や, つぎ, つぐ]",[つ],[つぐ],"[つ, つぐ, あ]"
1,唖,NaN,"[mute, dumb]","[あ, あく]",['おし'],[],[おし],[おし],"[おし, おし, あ, あく]"
2,娃,NaN,[beautiful],"[あ, あい, わ]",['うつく.しい'],[い],[うつく],[うつくしい],"[うつく, うつくしい, あ, あい, わ]"
3,阿,1126.0,"[Africa, flatter, fawn upon, corner, nook, rec...","[あ, お]","['おもね.る', 'くま']","[ほとり, あず, あわ, おか, きた, な]","[おもね, くま]","[おもねる, くま]","[おもね, くま, おもねる, くま, あ, お]"
4,哀,1715.0,"[pathetic, grief, sorrow, pathos, pity, sympat...",[あい],"['あわ.れ', 'あわ.れむ', 'かな.しい']",[],"[かな, あわ]","[かなしい, あわれ, あわれむ]","[かな, あわ, かなしい, あわれ, あわれむ, あい]"


In [9]:
def unvoice(char):
    if not ('\u3041' <= char <= '\u3096') or len(char) > 1 or not unicodedata.decomposition(char):
        return char
    return unicodedata.normalize('NFD', char)[0]

In [48]:
# if prons[-1] == ん: prons = [prons,prons[:-1]]
# if 
def recurs(kanji, word, pron_list = [], done = False):
    if len(pron_list) == len(kanji) or len(word) == 0: # check for 々 and repeat past syllable (unsounded)
        return pron_list, False
    cur = kanji[len(pron_list)]
    if re.match("""[\u3040-\u30ff]""", cur) != None: #should we romkan to hiragana?? ALSO HYPHEN
        if cur != word[0]:
            return pron_list, False
        prons = [cur]
    elif cur == '々':
        if unvoice(pron_list[-1]) == unvoice(word[:len(pron_list[-1])]): # Sou Zou -- didn't catch!!
            pron_list.append(word[:len(pron_list[-1])])
            return pron_list, len(kanji) == len(pron_list)
        else:
            return None, False
    else:
        prons = list(set(char_ser.get(cur, []))) # [char_ser[cur]] ###.sort(key=len, reverse=True)
        if prons == []:
            return None, False
    for pron in prons:        # GET RID OF THIS?
        if pron[-1] == 'ん'and pron != 'ん':
            if isinstance(prons,str):
                prons = [pron, pron[:-1]]
            elif pron[:-1] not in prons:
                prons.append(pron[:-1])
    for pron in prons:
        wordmatch = word[:len(pron)] ###-1
        wordmatch = unvoice(wordmatch[0]) + wordmatch[1:]
        words = [wordmatch, word[:len(pron)]] ### -1
        #print(kanji, word, pron_list, words)
        if pron in words:
            pron_list.append(word[:len(pron)])
            word = word[len(pron):]
            if len(pron_list) == len(kanji) and len(word) == 0:
                return pron_list, True
            new_pron_list, done = recurs(kanji,word,pron_list,done)
            if done:
                return new_pron_list, done
            word = pron_list.pop() + word
    return None, False

# update with unvoice the first syllable of word


# PRONUNCIATION DOES NOT MATCH ON or KUN YOMIKATA... MOSTLY DROPPING LAST N??? -------------

In [13]:
list(set(char_ser['擽'])) #中腹 三日月
#'boy'[1:]
#recurs('相変わらず','あいかわらず',[])
#print(recurs('指輪','ゆびわ',[]))
#char_ser['一']#.sort(key=len, reverse=True)    # EITHER INCLUDE NANORI OR FIX THE ABOVE

['うつ', 'りゃく', 'れき', 'くすぐ', 'う', 'ふき', 'くすぐる', 'らく']

In [49]:
true_cnt = 0
false_cnt = 0
false_dict = {}
true_dict = {}

for _, line in propernouns.iterrows():
    kanji = line.kanji
    word = line.pronunciation
    prons, _ = recurs(kanji,word,[])
    if prons == None:
        false_cnt += 1
        false_dict[kanji] = word
    else:
        true_cnt += 1
        true_dict[kanji] = prons
print(true_cnt, false_cnt) #~8% are irregular for jlpt N1 (jouyou vocab)
print(false_dict)
# 124,660 + 76,654 = 201,314
# 38% irregular. Much higher than ~8% irregular for everyday vocab.

124660 76654
{'下永添': 'しもながそい', '下白銀': 'しもしろがね', '郡寿': 'くにひさ', '旧ハッサム邸': 'きゅうはっさむてい', '記世': 'ふみよ', '季遥子': 'きよこ', '王寿': 'みほぎ', '呉海': 'くれみ', '結久': 'むく', '葛城ゴルフ場': 'かつらぎごるふじょう', '銀羽': 'ぎなば', '栄周': 'ひでひろ', '琴之介': 'きんのすけ', '馨曜子': 'かよこ', '喜望子': 'きみこ', '寒河江中央工業団地': 'さがえちゅうおうこうぎょうだんち', '競': 'わたなべ', '繋': 'つなぎ', '環唯': 'たまゆい', '君姫': 'きみひ', '育利': 'やすとし', '月読宮': 'つきよみのみや', '銀星': 'かなほ', '加藤忠廣': 'かとうただひろ', '梶井純': 'かじいしゅん', '右左郎': 'うさぶろう', '愛理澄': 'まりと', '下千間谷地': 'しもせんげやち', '音瀬ノ鼻': 'おとせのはな', '安達俊雄': 'あだちとしお', '携': 'すがる', '佳津史': 'かつふみ', '允昭': 'よしあき', '紀洲谷': 'きしゅうや', '希水男': 'きみお', '於兎牛山': 'おそうしやま', '戸刈': 'とがり', '訓高': 'のりたか', '花霞': 'あやか', '桑納': 'かんのう', '羽布ダム': 'はぶだむ', '久幸': 'ひさゆき', '鑑理': 'あきまさ', '久保野谷': 'くぼのや', '潔丈': 'きよたけ', '愛淑': 'えすぎ', '茄子川': 'なすびがわ', '功之丞': 'こうのすけ', '宏太郎': 'あつたろう', '葛老山トンネル': 'かつろうやまとんねる', '橋本勝三郎': 'はしもとかつさぶろう', '訓郎': 'のりお', '旭凰': 'ぎょくほう', '下根古谷': 'しもねこや', '閑史': 'やすちか', '衣斐': 'えみ', '慶太': 'よしひろ', '右谷': 'みぎや', '好量': 'よしかず', '伊谷': 'いや', '海老ケ島': 'えびがしま', '英史': 'ひでぶみ', '元伝': 'もとつぐ', '亜朱紗': '

In [50]:
#true_dict['言う']
#import csv

#my_dict = {"test": 1, "testing": 2}

#with open('falsedict.csv','w') as f:
#    w = csv.writer(f)
#    w.writerows(false_dict.items())

#with open('truedict.csv','w') as f:
#    w = csv.writer(f)
#    w.writerow(['Kanji','Pronunciation'])
#    w.writerows(true_dict.items())

import pickle

#true_dict.to_pickle()
pickle.dump( true_dict, open( "true.p", "wb" ) )

In [43]:
true_dict.keys() & false_dict.keys()

{'華都子',
 '記世',
 '右衛門佐',
 '亜里美',
 '銀羽',
 '後上',
 '一坊寺',
 '希華',
 '康仁',
 '江浦',
 '永珠',
 '円居',
 '兼二',
 '競',
 '繋',
 '一詠',
 '起基',
 '貴章',
 '銀星',
 '永善',
 '音治',
 '宇田',
 '形部',
 '久津間',
 '慶佳',
 '鴨東',
 '亜望',
 '櫛谷',
 '紀帆',
 '郁己',
 '久幸',
 '王子谷',
 '詠子',
 '光島',
 '椛島',
 '翁田',
 '浩郎',
 '衣斐',
 '右谷',
 '伊谷',
 '英史',
 '英江',
 '儀三',
 '給田',
 '好本',
 '教佳',
 '一入',
 '公威',
 '鴨飛田',
 '結斐',
 '鴬出',
 '絢正',
 '英臣',
 '嘉寿',
 '貫渡',
 '佳冶',
 '偉',
 '孝爾',
 '屋原',
 '一徳',
 '公受',
 '吉野建',
 '敬徳',
 '河道',
 '栗原町',
 '幸心',
 '原垣内',
 '記美',
 '垢木',
 '井二',
 '更科',
 '系井',
 '県町',
 '郁保',
 '華凪',
 '公通',
 '吉恵',
 '江島',
 '古明地',
 '光正',
 '解子',
 '岳峰',
 '権現岳',
 '遠竹',
 '悦治',
 '光興',
 '吉本隆明',
 '岡明',
 '桂島',
 '敢太郎',
 '佳音',
 '一社',
 '演義',
 '久之',
 '金田',
 '栄達',
 '角井',
 '羽美',
 '輝葉',
 '海錫',
 '岡添',
 '永尾',
 '幸磨',
 '円城',
 '佳一郎',
 '宏修',
 '栄開',
 '安口',
 '華菜',
 '康征',
 '葵梨',
 '丸子',
 '安谷',
 '詠木',
 '夏奈',
 '康淳',
 '葵水',
 '慧衣',
 '橘高',
 '境谷',
 '華香',
 '佳保理',
 '海保',
 '金治郎',
 '荻本',
 '幸矢',
 '沓掛',
 '我孫子',
 '愛唯',
 '角雄',
 '映五',
 '幸和',
 '幸見',
 '敬節',
 '敬道',
 '儀太郎',
 '椛山',
 '午王',
 '庵原町',
 '公志',
 

In [1]:
false_dict.keys() | true_dict.keys()

NameError: name 'false_dict' is not defined

In [44]:
both_keys = true_dict.keys() & false_dict.keys()

In [46]:
len(both_keys)

10679

In [45]:
#jlpt[jlpt.Kanji.str.contains('亜')]
#jlpt[jlpt.Kanji.isnull()]
print(recurs('十一人','じゅういちにん',[]))
print(recurs('一人','ひとり',[]))
#print(jlpt)

(['じゅう', 'いち', 'にん'], True)
(['ひと', 'り'], True)
